In [14]:
import geopandas as gpd

blocks_land_use_gdf = gpd.read_parquet('./../data/blocks.parquet')[['geometry', 'land_use']]
blocks_land_use_gdf.head()

,geometry,land_use
id,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",None
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",transport
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",recreation
3,"POLYGON ((355099.099 6623847.765, 355074.808 6...",residential
4,"POLYGON ((352766.168 6621954.748, 352744.412 6...",recreation


In [15]:
from blocksnet.preprocessing.feature_engineering import generate_geometry_features

blocks_features_gdf = generate_geometry_features(blocks_land_use_gdf, aspect_ratios=True)
blocks_features_gdf.head()

2025-03-14 01:26:47.801 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:34 - Calculating usual features.
2025-03-14 01:26:48.221 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_aspect_ratios:57 - Calculating radiuses.
100%|██████████| 16320/16320 [00:00<00:00, 32277.43it/s]


,geometry,x,y,area,length,corners_count,aspect_ratio
id,,,,,,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",354523.996322,6.625731e+06,8.044667e+05,6143.314507,180,2.194869
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",355410.584682,6.623480e+06,2.317313e+04,1305.400332,53,15.785382
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",353810.130041,6.625813e+06,3.630058e+05,2744.750098,88,1.107639
3,"POLYGON ((355099.099 6623847.765, 355074.808 6...",355194.637468,6.624275e+06,1.964145e+05,2588.389797,77,2.632711
4,"POLYGON ((352766.168 6621954.748, 352744.412 6...",353157.203348,6.622549e+06,1.781752e+06,5826.501550,156,1.864688


In [13]:
blocks_gdf = blocks_features_gdf.join(blocks_land_use_gdf[['land_use']])
blocks_gdf.head()

,geometry,x,y,area,length,corners_count,aspect_ratio,land_use
id,,,,,,,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",354523.996322,6.625731e+06,8.044667e+05,6143.314507,180,2.194869,None
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",355410.584682,6.623480e+06,2.317313e+04,1305.400332,53,15.785382,transport
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",353810.130041,6.625813e+06,3.630058e+05,2744.750098,88,1.107639,recreation
3,"POLYGON ((355099.099 6623847.765, 355074.808 6...",355194.637468,6.624275e+06,1.964145e+05,2588.389797,77,2.632711,residential
4,"POLYGON ((352766.168 6621954.748, 352744.412 6...",353157.203348,6.622549e+06,1.781752e+06,5826.501550,156,1.864688,recreation


In [ ]:
from blocksnet.relations import generate_adjacency_graph

adjacency_graph = generate_adjacency_graph(blocks_land_use_gdf, 100)

2025-03-14 01:09:42.885 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:8 - Generating nodes.
2025-03-14 01:09:42.891 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:13 - Generating edges.
2025-03-14 01:09:45.001 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:32 - Adjacency graph successfully generated: 16320 nodes, 16202 edges


## Optimization

In [21]:
adjacency_graph